# 03 - Déploiement et Interface Gradio

Ce notebook montre comment déployer le modèle via une interface **Gradio** :
- Chargement des préprocesseurs (TF-IDF, OneHot)
- Chargement des modèles entraînés
- Construction de la fonction de prédiction
- Mise en place de l'interface interactive


## Import des librairies

In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
import gradio as gr
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

## Préprocesseurs

In [ ]:
DATA_PATH = os.path.join('data', 'fiverr_gigs', 'fiverr-data-gigs-cleaned.csv')
df = pd.read_csv(DATA_PATH, encoding='latin-1', low_memory=False)
vectoriseur_titre = TfidfVectorizer(max_features=2000, stop_words='english')
vectoriseur_titre.fit(df['Title'].fillna(''))
encodeur_niveau = OneHotEncoder(sparse=False, handle_unknown='ignore')
encodeur_niveau.fit(df['Seller Level'].fillna('Inconnu').values.reshape(-1, 1))

## Modèles

In [ ]:
MODELS_DIR = 'models'
modele_reg = joblib.load(os.path.join(MODELS_DIR, 'mor_rf.pkl'))
modele_clf = joblib.load(os.path.join(MODELS_DIR, 'level_clf.pkl'))

## Fonction de prédiction

In [ ]:
def predict(titre_service, niveau_vendeur, nombre_avis):
    vec_titre = vectoriseur_titre.transform([titre_service])
    vec_niveau = encodeur_niveau.transform([[niveau_vendeur]])
    vec_avis = sparse.csr_matrix(np.array([[int(nombre_avis)]]))
    X = sparse.hstack([vec_titre, sparse.csr_matrix(vec_niveau), vec_avis], format='csr')
    prix_pred, note_pred = modele_reg.predict(X)[0]
    idx_niveau = modele_clf.predict(X)[0]
    niveau_pred = encodeur_niveau.categories_[0][idx_niveau]
    prix = round(float(prix_pred), 2)
    note = round(float(note_pred), 2)
    return prix, note, niveau_pred

## Interface Gradio

In [ ]:
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Textbox(label="Titre du service", placeholder="Ex: Je vais développer votre site web..."),
        gr.Dropdown(label="Niveau du vendeur", choices=encodeur_niveau.categories_[0].tolist()),
        gr.Number(label="Nombre d'avis", value=0)
    ],
    outputs=[
        gr.Number(label="Prix prédit (EUR)"),
        gr.Number(label="Note moyenne prédite"),
        gr.Textbox(label="Niveau du vendeur prédit")
    ],
    title="Démo Prédiction Fiverr",
    description="Prédiction du prix, de la note et du niveau du vendeur via un modèle ML et une interface Gradio."
)

iface.launch()